<a href="https://colab.research.google.com/github/supuni9622/Test-Google-Colab/blob/main/Keras(dataset5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
cd /content/drive/MyDrive/4th Year Research/M3/Test Implementation/Test-Google-Colab/TestV2/TFID

/content/drive/MyDrive/4th Year Research/M3/Test Implementation/Test-Google-Colab/TestV2/TFID


In [3]:
import pandas as pd
import numpy as np
from keras import regularizers, optimizers
from keras.layers import TextVectorization
from keras.layers import Embedding, Dense, Dropout, Input, LSTM, GlobalMaxPool1D
from keras.models import Sequential
from keras.initializers import Constant
import tensorflow as tf
import spacy

In [8]:
# download and import the large english model.
!python -m spacy download en_core_web_lg
import en_core_web_lg

nlp = en_core_web_lg.load()
Vectorizer = TextVectorization()

     |████████████████████████████████| 827.9 MB 1.2 MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=38b74cad6bd41c6b80a7a8e6e4c2b4f73d02461311f96e10ca177b2122173e22
  Stored in directory: /tmp/pip-ephem-wheel-cache-lxthrc5b/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [9]:
df = pd.read_csv('Lyrical_data_5.csv',encoding='cp1252')
df = df.dropna(how="any").reset_index(drop=True)

In [10]:
from numpy import random
import gensim
import nltk
nltk.download('stopwords')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re

df = df.reset_index(drop=True)
REPLACE = re.compile('[^a-zA-Z]')
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE.sub(' ', text)
    #text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
df['Chorus'] = df['Chorus'].apply(clean_text)

In [22]:
possible_labels = df.Emotion.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'Angry': 4, 'Calm': 3, 'Happy': 5, 'Inspiration': 0, 'Love': 1, 'Sad': 2}

In [23]:
df['label'] = df.Emotion.replace(label_dict)

In [24]:
#fit the vectorizer on the text and extract the corpus vocabulary
Vectorizer.adapt(df.Chorus.to_numpy())
vocab = Vectorizer.get_vocabulary()

In [25]:
#generate the embedding matrix
num_tokens = len(vocab)
embedding_dim = len(nlp('The').vector)
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for i, word in enumerate(vocab):
    embedding_matrix[i] = nlp(str(word)).vector

In [26]:
#Load the embedding matrix as the weights matrix for the embedding layer and set trainable to False
Embedding_layer=Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=Constant(embedding_matrix),
    trainable=False)

In [27]:
#build the model.  This is a bigger one, but it works well on this problem.
model = Sequential()
model.add(Input(shape=(1,), dtype=tf.string))
model.add(Vectorizer)
model.add(Embedding_layer)
model.add(LSTM(25, return_sequences=True))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.5))
model.add(Dense(32, activation='tanh', 
                kernel_regularizer = regularizers.l1_l2(l1=1e-5, l2=1e-4)))
model.add(Dropout(0.5))
model.add(Dense(32, activation='tanh', 
                kernel_regularizer = regularizers.l1_l2(l1=1e-5, l2=1e-4)))    
model.add(Dense(1))

model.compile(optimizer = 'adam', loss = 'mean_absolute_error', metrics = None)

print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding_1 (Embedding)     (None, None, 300)         1052700   
                                                                 
 lstm_3 (LSTM)               (None, None, 25)          32600     
                                                                 
 global_max_pooling1d_3 (Glo  (None, 25)               0         
 balMaxPooling1D)                                                
                                                                 
 dropout_6 (Dropout)         (None, 25)                0         
                                                                 
 dense_9 (Dense)             (None, 32)               

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Song_ID     800 non-null    int64 
 1   Song_Title  800 non-null    object
 2   Chorus      800 non-null    object
 3   Emotion     800 non-null    object
 4   label       800 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 31.4+ KB


In [32]:
#fit the model
model.fit(df.Chorus,
          df.label,
          batch_size = 10,
          epochs = 50,
          validation_split=0.2,
          shuffle = True
          )

Epoch 1/50
64/64 [==============================] - 2s 30ms/step - loss: 0.5276 - val_loss: 1.4162
Epoch 2/50
64/64 [==============================] - 2s 28ms/step - loss: 0.5238 - val_loss: 1.3876
Epoch 3/50
64/64 [==============================] - 2s 28ms/step - loss: 0.5027 - val_loss: 1.3940
Epoch 4/50
64/64 [==============================] - 2s 29ms/step - loss: 0.5254 - val_loss: 1.3868
Epoch 5/50
64/64 [==============================] - 2s 29ms/step - loss: 0.4777 - val_loss: 1.4168
Epoch 6/50
64/64 [==============================] - 2s 28ms/step - loss: 0.5167 - val_loss: 1.4302
Epoch 7/50
64/64 [==============================] - 2s 29ms/step - loss: 0.4932 - val_loss: 1.4103
Epoch 8/50
64/64 [==============================] - 2s 28ms/step - loss: 0.5154 - val_loss: 1.3834
Epoch 9/50
64/64 [==============================] - 2s 28ms/step - loss: 0.4836 - val_loss: 1.3980
Epoch 10/50
64/64 [==============================] - 2s 29ms/step - loss: 0.4837 - val_loss: 1.3619
Epoch 11/

In [33]:
accr = model.evaluate(df.Chorus,df.label)
accr

25/25 [==============================] - 0s 16ms/step - loss: 0.5041


0.5041497349739075

[Pretrained Word Embeddings using SpaCy and Keras TextVectorization](https://towardsdatascience.com/pretrained-word-embeddings-using-spacy-and-keras-textvectorization-ef75ecd56360)